# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [2]:
# Importing necessary libraries
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# Step 1: Load the dataset
df = pd.read_csv('tic-tac-toe.csv')

# Step 2: Inspect the dataset
print("First 5 rows of the dataset:")
print(df.head())
print("\n")  # Adding space between outputs

# Step 3: Convert categorical values to numeric
label_encoder = LabelEncoder()
df_encoded = df.apply(label_encoder.fit_transform)

# Print after converting categorical values to numeric
print("First 5 rows after converting categorical values to numeric:")
print(df_encoded.head())
print("\n")  # Adding space between outputs

# Step 4: Separate the inputs and output
X = df_encoded.drop('class', axis=1)  # Features (input columns)
y = df_encoded['class']  # Output (class column)

# Step 5: Normalize the input data
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

# Display the processed dataset
print("First 5 rows of normalized input data:")
print(X_normalized[:5])
print("\n")  # Adding space between outputs
print("First 5 labels (output):")
print(y[:5])


0       True
1       True
2       True
3       True
4       True
       ...  
953    False
954    False
955    False
956    False
957    False
Name: class, Length: 958, dtype: bool
[['x' 'x' 'x' ... 'x' 'o' 'o']
 ['x' 'x' 'x' ... 'o' 'x' 'o']
 ['x' 'x' 'x' ... 'o' 'o' 'x']
 ...
 ['o' 'x' 'o' ... 'x' 'o' 'x']
 ['o' 'x' 'o' ... 'x' 'o' 'x']
 ['o' 'o' 'x' ... 'o' 'x' 'x']]
[[2 2 2 ... 2 1 1]
 [2 2 2 ... 1 2 1]
 [2 2 2 ... 1 1 2]
 ...
 [1 2 1 ... 2 1 2]
 [1 2 1 ... 2 1 2]
 [1 1 2 ... 1 2 2]]


## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [3]:
# your code here

# Importing necessary libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

# 1. Split the dataset into training and testing data
# Assuming you already have your dataset loaded as X (features) and y (labels)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 2. Create a Sequential model
model = Sequential()

# 3. Add layers to the model
# Instead of using input_dim, use the Input layer for specifying the input shape
model.add(Input(shape=(X_train.shape[1],)))  # Input layer
model.add(Dense(128, activation='relu'))  # First hidden layer
model.add(Dense(64, activation='relu'))  # Second hidden layer
model.add(Dense(32, activation='relu'))  # Third hidden layer

# Output layer
model.add(Dense(3, activation='softmax'))  # Output layer for 3 classes (modify as needed)

# 4. Compile the model using 'adam' optimizer and 'sparse_categorical_crossentropy' loss function
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 5. Fit the model with the training data
history = model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=1, validation_data=(X_test, y_test))

# 6. Evaluate the model using the test data
test_loss, test_accuracy = model.evaluate(X_test, y_test)

# 7. Print the results
print(f"Training Accuracy (Last Epoch): {history.history['accuracy'][-1]:.4f}")
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

# 8. Save the model with a valid extension
model.save("tic-tac-toe.keras")

# 9. Print model summary to inspect architecture
model.summary()

/Users/cindymartinezgrullon/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/cindymartinezgrullon/Library/Python/3.9/lib/python/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


ValueError: Invalid dtype: object

## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [ ]:
# your code here

# Import necessary libraries
from tensorflow.keras.models import load_model
import numpy as np

# 1. Load the saved model
model = load_model("tic-tac-toe.keras")

# 2. Make predictions on the test data
# Let's select a few random rows from the test dataset
random_indices = np.random.choice(len(X_test), size=5, replace=False)

# Use .iloc to select the rows correctly from the DataFrame
X_random = X_test.iloc[random_indices]

# 3. Predict the class labels for the random test samples
predictions = model.predict(X_random)

# 4. Print the predictions and the corresponding true labels
for i, index in enumerate(random_indices):
    true_label = y_test.iloc[index]  # If y_test is also a DataFrame, use .iloc to access it
    predicted_label = np.argmax(predictions[i])  # Get the index of the highest probability
    print(f"Sample {i+1}:")
    print(f"True label: {true_label}, Predicted label: {predicted_label}")
    print(f"Prediction probabilities: {predictions[i]}")
    print("-" * 30)

## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [ ]:
# your code here

# Import necessary libraries
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# 1. Create a new Sequential model
model_improved = Sequential()

# 2. Add more layers to the model
model_improved.add(Input(shape=(X_train.shape[1],)))  # Input layer
model_improved.add(Dense(128, activation='relu'))  # First hidden layer
model_improved.add(Dense(64, activation='relu'))  # Second hidden layer
model_improved.add(Dense(32, activation='relu'))  # Third hidden layer
model_improved.add(Dense(16, activation='relu'))  # Added more layers (fourth hidden layer)

# Output layer
model_improved.add(Dense(3, activation='softmax'))  # Output layer for 3 classes (modify as needed)

# 3. Adjust the learning rate for the Adam optimizer
learning_rate = 0.0005  # Example of adjusting the learning rate
optimizer = Adam(learning_rate=learning_rate)

# 4. Compile the model
model_improved.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 5. Fit the model with the training data (increase the number of epochs)
early_stopping = EarlyStopping(monitor='val_loss', patience=10)  # Early stopping callback
history_improved = model_improved.fit(X_train, y_train, epochs=100, batch_size=32, verbose=1, 
                                      validation_data=(X_test, y_test), callbacks=[early_stopping])

# 6. Evaluate the model using the test data
test_loss_improved, test_accuracy_improved = model_improved.evaluate(X_test, y_test)

# 7. Print the results
print(f"Improved Training Accuracy (Last Epoch): {history_improved.history['accuracy'][-1]:.4f}")
print(f"Improved Test Loss: {test_loss_improved:.4f}")
print(f"Improved Test Accuracy: {test_accuracy_improved:.4f}")

# 8. Save the improved model
model_improved.save("tic-tac-toe-improved.keras")

**Which approach(es) did you find helpful to improve your model performance?**


The most helpful approach was adjusting the learning rate and increasing the number of epochs.